In [5]:
import numpy as np
from abc import ABC, abstractmethod

In [72]:
# how to create animal:

# pig = Animal()

# what I want to happen:

# animal.target_func = get_target_func(*species_data.loc[i, 'Target':'Target_pos'])
# animal.effect_func = get_effect_func(species, lvl, *data.loc['Effect':'Effect_repeat'])

# animal.target_func(own_team, opp_team/shop) --> [targets]
# animal.effect_func([targets], own_team, opp_team/shop, n_triggers) --> does the effect

In [8]:
class TargetFunction(ABC):
    @abstractmethod
    def __init__():
        pass
    
    def __call__(self, gamestate, in_team1, triggerers = []):
        if isinstance(self, Triggerer):
            return triggerers
        
        if isinstance(gamestate, Battle):
            own_team = gamestate.team1 if in_team1 else gamestate.team2
            opp_team = gamestate.team2 if in_team1 else gamestate.team1
            shop_team = ShopTeam()
        else:
            own_team = gamestate.team1
            opp_team = Team([])
            shop_team = gamestate.team2
        
        return self.function(self, own_team, opp_team, shop_team)    
        
class AllFriends(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            if self.owner.hp <= 0:
                return own_team.pets
            else:
                return own_team.exclude(self.owner.pos)
        self.owner = owner
        self.function = function
        
class AllFriendsAhead(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return own_team.pets[:self.owner.pos]
        self.owner = owner
        self.function = function
        
class AllNoFood(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return [pet for pet in own_team if pet.status.name == 'NoStatus']
        self.owner = owner
        self.function = function
        
class AllPets(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return own_team.pets + opp_team.pets
        self.owner = owner
        self.function = function
        
class AllShopPets(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return shop_team.pets
        self.owner = owner
        self.function = function
        
class AllStrawberry(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return [pet for pet in own_team if pet.status.name == 'Strawberry']
        self.owner = owner
        self.function = function
        
class DifferentTierFriends(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            tiers = [pet.tier for pet in own_team]
            inds = range(len(own_team) - 1, -1, -1)
            return [own_team[i] for i in inds if tiers[i] not in tiers[i+1:own_team.size()]]
        self.owner = owner
        self.function = function
        
class EachLevel(TargetFunction):
    # TODO: decide how to tiebreak
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return []
        self.owner = owner
        self.function = function
        
class EnemyPet(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return [opp_team[i] for i in target_pos if -len(opp_team)<=i<len(opp_team)]
        self.owner = owner
        self.function = function
        
class FaintedFriend(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return [pet for pet in own_team if pet.trigger_dict['Faint'] > 0]
        self.owner = owner
        self.function = function
        
class FedFriend(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return [pet for pet in own_team if pet.trigger_dict['EatsShopFood'] > 0]
        self.owner = owner
        self.function = function
        
class HealthiestFriend(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            excluded = own_team.exclude(self.owner.pos)
            if len(excluded) == 0:
                return []
            return [excluded[np.argmax([pet.hp for pet in excluded])]]
        self.owner = owner
        self.function = function
        
class HealthiestEnemy(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            if len(opp_team) == 0:
                return []
            return [opp_team[np.argmax([pet.hp for pet in opp_team])]]
        self.owner = owner
        self.function = function
        
class HighestTierFaintEnemy(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            if len(opp_team) == 0:
                return []
            faint_tiers = [pet.tier if any([abl.trigger == 'Faint' for abl in pet.abilities[pet.lvl]]) else -1 
                             for pet in opp_team]
            ind = np.argmax(faint_tiers)
            if faint_tiers[ind] == -1:
                return []
            else:
                return [opp_team[ind]]
        self.owner = owner
        self.function = function
        
#TODO: penguin nerf
class HighLevelFriends(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return [pet for pet in own_team.exclude(self.owner.pos) if pet.lvl >= 2]
        self.owner = owner
        self.function = function
        
class NoTarget(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return []
        self.function = function
        
class RandomEnemy(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return list(np.random.choice(opp_team, size=min(n_targets, len(opp_team)), replace=False))
        self.owner = owner
        self.function = function
        
class RandomFriend(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            excluded = own_team if self.owner.hp <= 0 else own_team.exclude(self.owner.pos)
            return list(np.random.choice(excluded, size=min(n_targets, len(excluded)), replace=False))
        self.owner = owner
        self.function = function
        
class RandomFrozen(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            frozen = [pet for pet in shop_team.pets if pet.is_frozen]
            return list(np.random.choice(frozen, size=min(n_targets, len(frozen)), replace=False))
        self.owner = owner
        self.function = function
        
class RandomHurtAbility(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            hurt_abls = [pet for pet in own_team if pet.abilities[pet.lvl][0] == 'Hurt']
            return list(np.random.choice(hurt_abls, size=min(n_targets, len(hurt_abls)), replace=False))
        self.owner = owner
        self.function = function
        
class RandomStrawberry(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            excluded = own_team if self.owner.hp <= 0 else own_team.exclude(self.owner.pos)
            strawberries = [pet for pet in excluded if pet.status.name == 'Strawberry']
            return list(np.random.choice(strawberries, size=min(n_targets, len(strawberries)), replace=False))
        self.owner = owner
        self.function = function
        
class RelativeEnemy(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return [opp_team[self.owner.pos + i] for i in target_pos if self.owner.pos+i < len(opp_team)]
        self.owner = owner
        self.function = function
        
class RelativeFriend(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            inds = [self.owner.pos + i - 1 if (i>0 and self.owner.hp <= 0) else self.owner.pos + i for i in target_pos]
            return [own_team[i] for i in inds if i in range(len(own_team))]
                    
        self.owner = owner
        self.function = function
        
class RelativePets(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            all_pets = opp_team.pets + own_team.pets
            own_pos = self.owner.pos + len(opp_team)
            # just follow the logic, idek
            inds = [own_pos+i-1 if (i>0 and self.owner.hp<=0) else own_pos+i for i in target_pos]
            return [all_pets[i] for i in inds if i in range(len(all_pets))]
            
        self.owner = owner
        self.function = function
        
class Self(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return [self.owner]
        self.owner = owner
        self.function = function
        
class SelectedPet(TargetFunction):
    def __init__(self, **kwargs):
        def function(self, own_team, opp_team, shop_team):
            return []
        self.function = function
        
class ShopPet(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return [shop_team.pets[i] for i in target_pos]
        self.owner = owner
        self.function = function
        
# TODO: should argmax include temp stats?
class StrongestFriend(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            excluded = own_team.exclude(self.owner.pos)
            if len(excluded) == 0:
                return []
            return [excluded[np.argmax([pet.hp + pet.atk for pet in excluded])]]
        self.owner = owner
        self.function = function
        
class SummonedFriend(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return [pet for pet in own_team.exclude(self.owner.pos) if pet.trigger_dict['Summoned'] > 0]
        self.owner = owner
        self.function = function
        
class SummonedEnemy(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return [pet for pet in opp_team if pet.trigger_dict['Summoned'] > 0]
        self.owner = owner
        self.function = function
        
class TeamPet(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return [own_team[i] for i in target_pos if i < len(own_team)]
        self.owner = owner
        self.function = function
        
class Triggerer(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        self.owner = owner
        
class UnfrozenShop(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            return [pet for pet in shop_team.pets if not pet.is_frozen]
        self.owner = owner
        self.function = function
        
class UnhealthiestEnemy(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            if len(opp_team) == 0:
                return []
            return [opp_team[np.argmin([pet.hp for pet in opp_team])]]
        self.owner = owner
        self.function = function

class UnhealthiestFriend(TargetFunction):
    def __init__(self, owner, n_targets, target_pos):
        def function(self, own_team, opp_team, shop_team):
            excluded = own_team.exclude(self.owner.pos)
            if len(excluded) == 0:
                return []
            return [excluded[np.argmin([pet.hp for pet in excluded])]]
        self.owner = owner
        self.function = function 
        
# Shop only
class RandomFriendlyPets(TargetFunction):
    def __init__(self, **kwargs):
        def function(self, own_team, opp_team, shop_team):
            return list(np.random.choice(own_team, size = min(len(own_team), kwargs['n_targets']), replace = False))
        self.function = function
        
class AllFriendlyPets(TargetFunction):
    def __init__(self, **kwargs):
        def function(self, own_team, opp_team, shop_team):
            return own_team.pets
        self.function = function
        
class PlayerShop(TargetFunction):
    def __init__(self, **kwargs):
        def function(self, own_team, opp_team, shop_team):
            return []
        self.function = function